## Matrace d'Incindince

Un indexation : 
[[Mots clés] ; [Documents]] = bool: Le mot clé est présent dans le document


Exemple d'écriture de matrice :
renard ET trompeur => 11000 (filtre sur la matrice booléenne où renard est index 1 et cigogne est index 2)
repas => 10110 ET NON cigogne => 11101 (tous les indexes sauf celui de cigogne)
=> 10100 résultat du "et" binaire

In [2]:
from pydantic import BaseModel, PrivateAttr
from typing import Any, Dict, List, Set, Self, Union, Callable
from abc import abstractmethod

import re
import pandas as pd

In [3]:
input_docs = {"docs": [
    {
        "name": "Le Corbeau et le Renard",
        "content": "Un corbeau tient un fromage dans son bec. Un renard rusé le flatte pour le tromper et lui faire lâcher son repas."
    },
    {
        "name": "Le Renard et le Bouc",
        "content": "Un renard et un bouc tombent dans un puits. Le renard parvient à tromper le bouc pour s’en sortir, puis l’abandonne au fond."
    },
    {
        "name": "Le Loup et l’Agneau",
        "content": "Un loup cherche un prétexte pour faire de l’agneau innocent son repas."
    },
    {
        "name": "Le Renard et la Cigogne",
        "content": "Un renard invite une cigogne à dîner mais lui sert un repas inadapté ; la cigogne se venge de la même manière."
    },
    {
        "name": "Le Loup devenu Berger",
        "content": "Un loup se déguise en berger pour tromper les moutons, mais il est vite démasqué et puni."
    }
]}

In [4]:
class Document: pass

class Expr:
    def __init__(self, term: str|List[str]|None = None):
        if isinstance(term, list):
            self.expression = term
        else:
            self.expression: List[str] = [term] if term else []

    def __or__(self, other: Self|str):
        if isinstance(other, Expr):
            return Expr([self, "_or", other])
        elif isinstance(other, str):
            return Expr(self.expression + ["_or", other])
        return NotImplemented
    
    def __and__(self, other: Self|str):
        if isinstance(other, Expr):
            return Expr([self, "_and", other])
        elif isinstance(other, str):
            return Expr(self.expression + ["_and", other])
        return NotImplemented
    
    def __str__(self) -> str:
        s = ""
        for item in self.expression:
            if isinstance(item, Expr):
                s += f"({str(item)})"
            elif item.startswith("_"):
                s += f" {item[1:]} "
            else:
                s += item
        return s
            
    def __repr__(self):
        return str(self)
    
e: Callable[[str], Expr] = lambda s: Expr(s)


class SearchIndex:
    @abstractmethod
    def union_search(self) -> Set[Any]:
        raise NotImplemented
    
    def search(self, expression: Expr | str) -> Set['Document']:
        """Recursively evaluates AND / OR operations."""
        if isinstance(expression, str):
            return self.union_search([expression])
        
        if len(expression.expression) == 1:  # Single keyword case
            return self.search(expression.expression[0])
        
        # Extract left, operator, and right expressions
        left, operator, right = expression.expression
        left_docs = self.search(left)
        right_docs = self.search(right)

        if operator == "_or":   # Perform OR (Union)
            return left_docs | right_docs
        elif operator == "_and":  # Perform AND (Intersection)
            return left_docs & right_docs

        return set()  # Default case (no match)

class IncidenceMatrix(pd.DataFrame, SearchIndex):
    def union_search(self, keywords: List[str]) -> List[str]:
        docs = set()
        for kw in keywords:
            if kw in self.columns:
                for doc_name in self.index:
                    if self.loc[doc_name, kw]:
                        docs.add(doc_name)
        return docs

class InvertedIndex(Dict[str, List['Document']], SearchIndex):
    def union_search(self, keywords: List[str]) -> List['Document']:
        # Assuming all input keywords are included
        docs = set()
        for kw in keywords:
            for doc in self.get(kw, []):
                docs.add(doc)
        return docs

In [5]:
class Document(BaseModel):
    _cached_keywords: Set[str] | None = PrivateAttr(None)

    name: str
    content: str

    def __hash__(self):
        return hash(self.name)

    @property
    def keywords(self) -> Set[str]:
        if not self._cached_keywords:
            spans = self.content.split()
            keywords = ["".join(re.findall(r'\w+', s)).lower() for s in spans if re.search(r'\w', s)]
            self._cached_keywords = set(keywords)
        return self._cached_keywords

class Corpus(BaseModel):
    docs: List[Document]

    @property
    def all_keywords(self) -> Set[str]:
        keywords = set()
        for doc in self.docs:
            keywords = keywords.union(doc.keywords)  # Implicit set.add()
        return keywords
    
    @property
    def document_names(self) -> List[str]:
        return [doc.name for doc in self.docs]
    
    @property
    def incidence_matrix(self) -> 'IncidenceMatrix':
        all_keywords: List[str] = list(self.all_keywords)
        return IncidenceMatrix(
            columns=all_keywords,
            index=self.document_names,
            data=[
                [
                    keyword in doc.keywords
                    for keyword in all_keywords
                ] 
                for doc in self.docs
            ]
        )
    
    @property
    def inverted_index(self) -> 'InvertedIndex':
        all_keywords: List[str] = list(self.all_keywords)
        index = InvertedIndex({keyword: [] for keyword in all_keywords})
        
        for keyword in all_keywords:
            for doc in self.docs:
                if keyword in doc.keywords:
                    index[keyword].append(doc)

        return index

In [6]:
documents = Corpus.model_validate(input_docs)

In [7]:
len(documents.inverted_index)

57

In [8]:
e("tromper") | e("loup") & "berger"

(tromper) or (loup and berger)

In [9]:
documents.incidence_matrix.search(e("tromper") | e("loup") & "berger")

{'Le Corbeau et le Renard', 'Le Loup devenu Berger', 'Le Renard et le Bouc'}

In [10]:
documents.inverted_index.union_search(["tromper", "loup"])

{Document(name='Le Corbeau et le Renard', content='Un corbeau tient un fromage dans son bec. Un renard rusé le flatte pour le tromper et lui faire lâcher son repas.'),
 Document(name='Le Loup devenu Berger', content='Un loup se déguise en berger pour tromper les moutons, mais il est vite démasqué et puni.'),
 Document(name='Le Loup et l’Agneau', content='Un loup cherche un prétexte pour faire de l’agneau innocent son repas.'),
 Document(name='Le Renard et le Bouc', content='Un renard et un bouc tombent dans un puits. Le renard parvient à tromper le bouc pour s’en sortir, puis l’abandonne au fond.')}

In [11]:
documents.incidence_matrix.union_search(["tromper", "loup"])

{'Le Corbeau et le Renard',
 'Le Loup devenu Berger',
 'Le Loup et l’Agneau',
 'Le Renard et le Bouc'}